In [1]:
import os
import sys
sys.path.append(os.path.join('..', '..'))

import datetime
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.models import Panel, Tabs
from bokeh.models.widgets import Div
from bokeh.plotting import figure
from bokeh.layouts import layout
from bokeh.models.tools import HoverTool

# custom libraries
from mikesnowflake.analysis.snowFlakeAnalysis import SnowFlakeAnalysis

output_notebook(hide_banner=True)

In [2]:
user = '######'
password = '######'

In [4]:
START_DATE = datetime.datetime(2018, 12, 1)
END_DATE = datetime.datetime(2020, 1, 14)
SFA = SnowFlakeAnalysis(START_DATE, END_DATE, user, password, excludeEtl=True, verbose=False)

In [5]:
tableNames = [
 'OX_CHAIN_METRICS_SUM_HOURLY_FACT', # confirming with sigmoid via slack on 01/15/2020
]

# collect queryType history for visualization analysis for each table in df
qtHistory = {}  # collector of query type history keyed by table name
for tableName in tableNames:
    qtHistory[tableName] = SFA.getQueryTypeHistory(tableName)

# show query type history for certain tables in question
p = {}  # collector of bokeh charts keyed by table name
for tableName in tableNames:
    p[tableName] = figure(width=1900, height=800, x_axis_type='datetime')
    for c in qtHistory[tableName].columns:
        p[tableName].line(qtHistory[tableName].index.tolist(), qtHistory[tableName][c], line_width=2, name=c, legend_label=c, line_color=SFA.queryTypeColors[c])
    p[tableName].title.text = 'Query Type History for %s' % tableName
    p[tableName].title.align = 'center'
    hover = HoverTool(tooltips=[('Date', '@x{%F}'),
                                ("Query Type", "$name"),
                                ("Hits", "@y{0,0}")],
                      formatters={'x': 'datetime'})
    p[tableName].add_tools(hover)
    p[tableName].legend.click_policy="hide"
tabSize = 5
subLists = [tableNames[i * tabSize:(i + 1) * tabSize] for i in range((len(tableNames) + tabSize - 1) // tabSize )]
tabs = []
for subList in subLists:
    tabs.append([Tabs(tabs=[Panel(child=p[tableName], title=tableName) for tableName in subList], background='whitesmoke')])
show(layout([Div(text="Query Type History </b>", style={'font-size': '150%'}, width=1500, height=25)],
            [tabs]))

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>